<a href="https://colab.research.google.com/github/jakubpiwowarski/Personal_protective_Equipment_detector./blob/main/PPE_detection_partly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.28

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

In [ ]:
pip install cvzone

In [ ]:
import cvzone
import math

In [ ]:
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install opencv-python-headless
!pip install google-colab

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# from IPython.display import Image
# try:
#   filename = take_photo()
#   print('Saved to {}'.format(filename))

#   # Show the image which was just taken.
#   display(Image(filename))
# except Exception as err:
#   # Errors will be thrown if the user does not have a webcam or if they do not
#   # grant the page permission to access it.
#   print(str(err))

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/101-Object_Detection/Pliki/Videos/ppe-3.mp4')

In [ ]:
model= YOLO('/content/drive/MyDrive/101-Object_Detection/Pliki/Project_3-PPE_Detection/ppe.pt')

In [ ]:
classNames = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone',
              'Safety Vest', 'machinery', 'vehicle']

In [ ]:
myColor = (0,0,255)

In [ ]:
while True:
    success, img = cap.read()
    results = model(img,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            #bounding box
            x1,y1,x2,y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2-x1, y2-y1
            bbox = int(x1), int(y1), int(x2), int(y2)
            # cvzone.cornerRect(img,(x1,y1,w,h))
            cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            #confidence
            conf = math.ceil((box.conf[0]*100))/100
            print(conf)

            #clas name
            cls = int(box.cls[0])
            CurrentClass = classNames[cls]
            if conf > 0.5:
              if CurrentClass == 'No-Hardhat' or CurrentClass == 'No-Mask' or CurrentClass == 'No-Safety Vest':
                myColor = (0,0,255)
              elif CurrentClass == 'Hardhat' or CurrentClass == 'Safety Vest' or CurrentClass == 'Mask':
                myColor = (0,255,0)
              else:
                  myColor = (255,0,0)
              cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)),scale=0.5,thickness=2
                                colorB=myColor,colorT=(255,255,255),colorR=myColor, offset=5)
              cv2.rectangle(img,(x1,y1),(x2,y2),myColor,3)

    cv2_imshow(img)
    cv2.waitKey(1)
